In [1]:
from shared.simulators import ConstantEffectSimulator, ModeratedEffectSimulator, DataSimulator, EffectSimulatorCollection
from shared.analyzers import AnalyzerCollection, WLSAnalyzer, IPWAnalyzer, SIPWAnalyzer, MetaAnalyzer
from shared.result_processors import Results
from tqdm import trange
import matplotlib.pyplot as plt
import copy

In [2]:
N = 50
T = 100
alpha = 0.05
n_reps = 100
weight_type = "ATE"

# Effect simulators
effect_simulators = EffectSimulatorCollection()
effect_simulators.add_effect_simulator(ConstantEffectSimulator("No Effect", N, 0., 0., 0.))
effect_simulators.add_effect_simulator(ConstantEffectSimulator("Shared Effect", N, 1., 1., 0.))
effect_simulators.add_effect_simulator(ConstantEffectSimulator("Distinct Effects", N, 1., 1., 0.3))
effect_simulators.add_effect_simulator(ModeratedEffectSimulator("Moderated Effects", N, 1., 1., 0.3, 1., 0.2))
effect_simulators.add_effect_simulator(ModeratedEffectSimulator("Noisy Moderated Effects", N, 1., 1., 0.3, 1., 0.2, epsilon_sd=0.5))

# Simulate effects once and fix
for effect_simulator in effect_simulators:
    effect_simulator.sample_params()

# Analyzers
analyzers = AnalyzerCollection()
wls_analyzer = WLSAnalyzer(name="WLS", alpha=alpha)
analyzers.add_analyzer(wls_analyzer)
analyzers.add_analyzer(MetaAnalyzer(wls_analyzer, name="Meta-WLS", alpha=alpha))

ipw_model_analyzer = IPWAnalyzer(name="IPW-Model", alpha=alpha, robust=False)
analyzers.add_analyzer(ipw_model_analyzer)
analyzers.add_analyzer(MetaAnalyzer(ipw_model_analyzer, name="Meta-IPW-Model", alpha=alpha))
ipw_robust_analyzer = IPWAnalyzer(name="IPW-Robust", alpha=alpha)
analyzers.add_analyzer(ipw_robust_analyzer)
analyzers.add_analyzer(MetaAnalyzer(ipw_robust_analyzer, name="Meta-IPW-Robust", alpha=alpha))

sipw_robust_analyzer = SIPWAnalyzer(name="SIPW-Model", alpha=alpha, robust=False)
analyzers.add_analyzer(sipw_robust_analyzer)
analyzers.add_analyzer(MetaAnalyzer(sipw_robust_analyzer, name="Meta-SIPW-Model", alpha=alpha))
sipw_robust_analyzer = SIPWAnalyzer(name="SIPW-Robust", alpha=alpha)
analyzers.add_analyzer(sipw_robust_analyzer)
analyzers.add_analyzer(MetaAnalyzer(sipw_robust_analyzer, name="Meta-SIPW-Robust", alpha=alpha))

ipw_model_analyzer = IPWAnalyzer(name="IPW-DR-Model", alpha=alpha, dr=True, robust=False)
analyzers.add_analyzer(ipw_model_analyzer)
analyzers.add_analyzer(MetaAnalyzer(ipw_model_analyzer, name="Meta-IPW-DR-Model", alpha=alpha))
ipw_dr_analyzer = IPWAnalyzer(name="IPW-DR-Robust", alpha=alpha, dr=True)
analyzers.add_analyzer(ipw_dr_analyzer)
analyzers.add_analyzer(MetaAnalyzer(ipw_dr_analyzer, name="Meta-IPW-DR-Robust", alpha=alpha))

sipw_dr_model_analyzer = SIPWAnalyzer(name="SIPW-DR-Model", alpha=alpha, dr=True, robust=False)
analyzers.add_analyzer(sipw_dr_model_analyzer)
analyzers.add_analyzer(MetaAnalyzer(sipw_dr_model_analyzer, name="Meta-SIPW-DR-Model", alpha=alpha))
sipw_dr_robust_analyzer = SIPWAnalyzer(name="SIPW-DR-Robust", alpha=alpha, dr=True)
analyzers.add_analyzer(sipw_dr_robust_analyzer)
analyzers.add_analyzer(MetaAnalyzer(sipw_dr_robust_analyzer, name="Meta-SIPW-DR-Robust", alpha=alpha))

# Simulation loop
data_simulator = DataSimulator(N, T).simulate_history()
results = Results(data_simulator, effect_simulators, analyzers, n_reps=n_reps)
for rep_idx in trange(n_reps):
    for effect_simulator in effect_simulators:
        data_simulator = data_simulator.simulate_a()
        data_simulator = data_simulator.simulate_effects(effect_simulator)
        for analyzer in analyzers:          
            # Inferences for theta
            theta_inferences = analyzer.get_theta_inferences(data_simulator)
            results = results.add_theta_inferences(theta_inferences, data_simulator.theta.copy(), rep_idx, effect_simulator.i, analyzer.i)
            
            # Inferences for user estimates
            user_inferences = analyzer.get_user_inferences(data_simulator)
            results = results.add_user_inferences(user_inferences, data_simulator.user_effects.copy(), rep_idx, effect_simulator.i, analyzer.i)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:11<00:00,  8.79it/s]


In [3]:
digits = 3
results = results.process(digits=digits)
results.user_results_df

WLS  \
Coefficient Metric       Effect Simulator                         
Intercept   Avg Bias     No Effect                0.178 (0.003)   
                         Shared Effect            0.182 (0.003)   
                         Distinct Effects         0.176 (0.003)   
                         Moderated Effects        0.479 (0.004)   
                         Noisy Moderated Effects  0.466 (0.004)   
            Avg Sq. Bias No Effect                        0.038   
                         Shared Effect                    0.039   
                         Distinct Effects                 0.037   
                         Moderated Effects                0.242   
                         Noisy Moderated Effects          0.231   
            MSE          No Effect                0.082 (0.002)   
                         Shared Effect            0.083 (0.001)   
                         Distinct Effects         0.081 (0.001)   
                         Moderated Effects        0.308 (0.004)   
                         Noisy Moderated Effects  0.299 (0.004)   
            Coverage     No Effect                0.857 (0.005)   
                         Shared Effect            0.857 (0.005)   
                         Distinct Effects         0.857 (0.005)   
                         Moderated Effects        0.626 (0.006)   
                         Noisy Moderated Effects  0.661 (0.007)   
            CI Length    No Effect                0.845 (0.000)   
                         Shared Effect            0.845 (0.000)   
                         Distinct Effects         0.846 (0.000)   
                         Moderated Effects        1.130 (0.001)   
                         Noisy Moderated Effects  1.161 (0.001)   

                                                       Meta-WLS  \
Coefficient Metric       Effect Simulator                         
Intercept   Avg Bias     No Effect                0.176 (0.003)   
                         Shared Effect            0.181 (0.003)   
                         Distinct Effects         0.174 (0.003)   
                         Moderated Effects        0.472 (0.004)   
                         Noisy Moderated Effects  0.454 (0.004)   
            Avg Sq. Bias No Effect                        0.034   
                         Shared Effect                    0.036   
                         Distinct Effects                 0.041   
                         Moderated Effects                0.244   
                         Noisy Moderated Effects          0.222   
            MSE          No Effect                0.055 (0.001)   
                         Shared Effect            0.057 (0.001)   
                         Distinct Effects         0.062 (0.001)   
                         Moderated Effects        0.275 (0.003)   
                         Noisy Moderated Effects  0.260 (0.004)   
            Coverage     No Effect                0.867 (0.005)   
                         Shared Effect            0.865 (0.005)   
                         Distinct Effects         0.839 (0.005)   
                         Moderated Effects        0.478 (0.008)   
                         Noisy Moderated Effects  0.559 (0.007)   
            CI Length    No Effect                0.697 (0.000)   
                         Shared Effect            0.697 (0.000)   
                         Distinct Effects         0.698 (0.000)   
                         Moderated Effects        0.912 (0.003)   
                         Noisy Moderated Effects  0.979 (0.003)   

                                                       IPW-Model  \
Coefficient Metric       Effect Simulator                          
Intercept   Avg Bias     No Effect                -0.001 (0.007)   
                         Shared Effect            -0.002 (0.005)   
                         Distinct Effects          0.009 (0.007)   
                         Moderated Effects         0.008 (0.006)   
                         Noisy M